In [1]:
# 0, 1)
import numpy as np
import pandas as pd
df = pd.read_csv("data.csv", sep = ',')
codes, platforms = pd.factorize(df["Platform"])
df["Platform_Code"] = codes
df[["Platform_Code","Engagement Rate"]].corr("spearman")


,Platform_Code,Engagement Rate
Platform_Code,1.000000,0.031382
Engagement Rate,0.031382,1.000000


In [2]:
#2) 
Follower_Bin = pd.qcut(df["Follower Count"],4,["Low", "Medium","High","Very High"])
df["Follower_Bin"] = Follower_Bin
categories = df.groupby(by="Follower_Bin")
high = categories.get_group("High")
high["Engagement Rate"].mean()

C:\Users\Admin\AppData\Local\Temp\ipykernel_32652\988132912.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  categories = df.groupby(by="Follower_Bin")


np.float64(0.08655032000000001)

In [3]:
value_vars = ['Follower Count', 'Posts Per Week', 'Ad Spend (USD)',
              'Conversion Rate', 'Engagement Rate', 'Campaign Reach']

melted_df = pd.melt(df, id_vars='Platform', value_vars=value_vars,
                    var_name='Metric', value_name='Value')

modes_df = (
    melted_df
    .groupby(['Platform', 'Metric'])['Value']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    .reset_index()
)

result = modes_df['Value'].sum()

print(result)

1641860.3451999999


In [4]:
#4) 
cats = df.groupby(by="Platform").apply(lambda x: x.sort_values(by="Follower Count")[::-1].head(3)).reset_index(drop=True)
print(cats)
print(cats["Follower Count"].min())

    Account ID         Username   Platform  Follower Count  Posts Per Week  \
0         2404           eric65   Facebook          999982               6   
1         7351     patricknoble   Facebook          997915               3   
2         1690      chavezjason   Facebook          997512               7   
3         8686  alexandersamuel  Instagram          999726               3   
4         3966         lrodgers  Instagram          999351               1   
5         2190           jbrown  Instagram          997844               5   
6         3040          toneill   LinkedIn          999055               4   
7         6360    andrewgregory   LinkedIn          998968               7   
8         2160     ashleycooper   LinkedIn          998925               6   
9         5839     edwardthomas     TikTok          999739               7   
10        4235    andradewesley     TikTok          999234               5   
11        2576     williamwyatt     TikTok          998623      

C:\Users\Admin\AppData\Local\Temp\ipykernel_32652\3513831787.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cats = df.groupby(by="Platform").apply(lambda x: x.sort_values(by="Follower Count")[::-1].head(3)).reset_index(drop=True)


In [5]:
#5) 
df["Conversion_Category"] = pd.qcut(df["Conversion Rate"],2, ["Low","High"])
temp = pd.pivot_table(df,values= ["Follower Count", "Campaign Reach"] ,index= "Platform", columns= "Conversion_Category",aggfunc="sum")
temp["Conversion Influence"] = (temp["Follower Count","High"] - temp["Follower Count","Low"])/(temp["Campaign Reach", "High"] +  temp["Campaign Reach", "Low"])
print(temp["Conversion Influence"].idxmax())


Twitter


C:\Users\Admin\AppData\Local\Temp\ipykernel_32652\3031103939.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  temp = pd.pivot_table(df,values= ["Follower Count", "Campaign Reach"] ,index= "Platform", columns= "Conversion_Category",aggfunc="sum")


In [6]:
#6)
temp_six = df.groupby(by="Platform").apply(lambda x: x.sort_values(by="Posts Per Week", ascending=False)).reset_index(drop=True)
temp_six["Ad Spend (USD) Window"] = temp_six["Ad Spend (USD)"].rolling(window=3).sum()
temp_six["Ad Spend (USD) Window"] = temp_six.groupby("Platform")["Ad Spend (USD)"].transform(lambda x: x.rolling(3).sum())
temp_six["Campaign Reach Window"] = temp_six.groupby("Platform")["Campaign Reach"].transform(lambda x: x.rolling(3).sum())
temp_six["Rolling Efficiency Ratio"] = temp_six["Campaign Reach Window"]/temp_six["Ad Spend (USD) Window"]
max = temp_six["Rolling Efficiency Ratio"].idxmax()
min = temp_six["Rolling Efficiency Ratio"].idxmin()
print("max:", df.loc[max, "Platform"])
print("min:", df.loc[min, "Platform"])

max: Facebook
min: LinkedIn


C:\Users\Admin\AppData\Local\Temp\ipykernel_32652\1799844455.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_six = df.groupby(by="Platform").apply(lambda x: x.sort_values(by="Posts Per Week", ascending=False)).reset_index(drop=True)


In [7]:
#7)
eng_rates = df.groupby(by= "Platform").apply(lambda x: x.sort_values(by="Engagement Rate",ascending=False).head(1)).reset_index(drop=True)
con_rates = df.groupby(by= "Platform").apply(lambda x: x.sort_values(by="Conversion Rate",ascending=False).head(1)).reset_index(drop=True)
rates = pd.merge(eng_rates,con_rates, on="Platform", how="outer", suffixes=('_eng', '_con'))
print("Количество совпадающих ников:", (rates["Username_eng"] == rates["Username_con"]).sum())


Количество совпадающих ников: 0


C:\Users\Admin\AppData\Local\Temp\ipykernel_32652\3407353575.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eng_rates = df.groupby(by= "Platform").apply(lambda x: x.sort_values(by="Engagement Rate",ascending=False).head(1)).reset_index(drop=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_32652\3407353575.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_rates = df.groupby(by= "Platform").apply(

In [8]:
#8)
mean_val = df["Conversion Rate"].mean()

df["Conversion_Category"] = df["Conversion Rate"].apply(
    lambda x: "High" if x > mean_val else "Low"
)
temp = pd.pivot_table(df,values= ["Follower Count"] ,index= "Platform", columns= "Conversion_Category",aggfunc="sum")
temp["Subs_Rate"] = temp["Follower Count","High"]/temp["Follower Count","Low"]
print("delta:", temp["Subs_Rate"].max() - temp["Subs_Rate"].min())
print("max:", temp["Subs_Rate"].idxmax(),',',"min:",temp["Subs_Rate"].idxmin())


delta: 0.17688741338715763
max: Twitter , min: Instagram
